In [1]:
import pandas as pd, numpy as np

In [ ]:
# pip install openpyxl

In [ ]:
def applytab(row):
    print('\t'.join(map(str,row.values)))

In [ ]:
df = pd.read_excel('/home/trishalaswain/Documents/AGS_PROJECTS/download_scripts_and_tars/CHS_ALL_JUNE/CHS-June-accuracy_sheets_combined.xlsx', index_col=False)

In [ ]:
df.facilityCode = df.facilityCode.astype(str)


In [ ]:
df = df[df.facilityCode=='1475']

In [ ]:
df2 = df.copy()

In [ ]:
df = pd.DataFrame()

In [ ]:
merge_df = pd.DataFrame()


In [2]:
import os

folder = "/home/trishalaswain/ClientData/SCP/SCP_JULY_ALL_SHEETS/"
# folder = "/home/trishalaswain/Downloads/Feb_01_to_28_tsv (1)/"
dfs = []
for filename in os.listdir(folder):
    print(filename)
    if filename.startswith('SCP-'):
        if filename.endswith("31.tsv"):
            print(os.path.join(folder, filename))
            dfs.append(pd.read_table(os.path.join(folder, filename),index_col=False))
        # if filename.endswith("15.xlsx"):
        #     print(os.path.join(folder, filename))
        #     dfs.append(pd.read_excel(os.path.join(folder, filename),index_col=False))
df = pd.concat(dfs)

# import os

# folder = "/home/trishalaswain/ClientData/SCP/SCP_JULY_ALL_SHEETS/"
# for filename in os.listdir(folder):
#     if filename.startswith('SCP-') and filename.endswith("31.tsv"):
#             print(os.path.join(folder, filename))
#             df = pd.read_table(os.path.join(folder, filename),index_col=False)

# merge_df = pd.concat([merge_df,df])

SCP-2361-TOURO INFIRMARY-LA-2023-07-01-2023-07-31-HCPCS.tsv
SCP-346-WESTERN ARIZONA REGIONAL MEDICAL CENTER-AZ-2023-07-01-2023-07-31-CPT.tsv
SCP-2392-WHITESBURG ARH HOSPITAL-KY-2023-07-01-2023-07-31-CPT.tsv
SCP-488965-ADVENTHEALTH FOUR CORNERS ER-FL-2023-07-01-2023-07-31-CM.tsv
SCP-1198-SEBASTIAN RIVER MEDICAL CENTER-FL-2023-07-01-2023-07-31.tsv
/home/trishalaswain/ClientData/SCP/SCP_JULY_ALL_SHEETS/SCP-1198-SEBASTIAN RIVER MEDICAL CENTER-FL-2023-07-01-2023-07-31.tsv
SCP-488739-MOUNT SINAI MEDICAL CENTER HIALEAH FREE STANDING EMERGENCY DEPARTMENT-FL-2023-07-01-2023-07-31-HCPCS.tsv
SCP-2586-NORTHERN LOUISIANA MEDICAL CENTER-LA-2023-07-01-2023-07-31-CM.tsv
SCP-120936-BIENVILLE MEDICAL CENTER-LA-2023-07-01-2023-07-31-CM.tsv
SCP-6074-LONESOME PINE HOSPITAL-ICP-VA-2023-07-01-2023-07-31-EANDM.tsv
SCP-482961-JEFFERSON RMC-URGENT CARE-AR-2023-07-01-2023-07-31-EANDM.tsv
SCP-6182-DICKENSON COMMUNITY HOSPITAL-ICP-VA-2023-07-01-2023-07-31-EANDM.tsv
SCP-1796-CHI ST VINCENT NORTH-AR-2023-07-01-2023-

In [4]:
df.id.to_excel('/home/trishalaswain/ClientData/Accounts.xlsx')

In [ ]:
df =  pd.read_excel('/home/trishalaswain/Downloads/CHS-4696-CHS - Tablespace 4696-2023-04-01-2023-04-21.xlsx',index_col=False).fillna('0')

In [ ]:
df

In [ ]:
# df[df.EANDM_Final.isna()].EANDM_Suggested.value_counts()
df[df.EANDM_Final.isna()].shape

In [ ]:
# pep_folder = '/home/vivekkumar/EandMTXT/docs_PEP/PEP/'

# enc_pcs_dict = {}
# # enc_req_dict = {}

# for subfoldername in os.listdir(pep_folder):
#     subfolder = os.path.join(pep_folder, subfoldername)
#     for filename in os.listdir(subfolder):
        
#         enc_id = int(filename[:filename.index('_')])
#         if filename.endswith('_pcs.xml'):
#             enc_pcs_dict[enc_id] = os.path.join(subfolder, filename)
# df.to_excel('/home/trishalaswain/Downloads/excel.xlsx')

In [ ]:
df.month.nunique()

In [ ]:
# df.to_csv('/home/vivekkumar/AccuracySheets/testt.csv', index=False)

In [ ]:
df.facilityName.value_counts()

In [ ]:
# df.columns

In [ ]:
from collections import OrderedDict
def get_accuracy(data_df, codetype):
    
    # display(data_df)
    # print(data_df.columns)
    
    accept = data_df[codetype+'_Accepted_Count'].sum()
    reject = data_df[codetype+'_Rejected_Count'].sum()
    add = data_df[codetype+'_Added_Count'].sum()
    
    d = OrderedDict()
    
    d['encounters'] = data_df.shape[0]
    
    d[codetype+'_accepted'] = accept
    d[codetype+'_rejected'] = reject
    d[codetype+'_added'] = add
    
    d[codetype+'_billed'] = accept+add
    
    if accept == 0:
        if add + reject == 0:
            d['precision'] = 1
            d['recall'] = 1
            d['f1'] = 1
        else:
            d['precision'] = 0.0
            d['recall'] = 0.0
            d['f1'] = 0.0
    else:
        d['precision'] = accept / (accept + reject)
        d['recall'] = accept / (accept + add)
        d['f1'] = 2 * d['precision'] * d['recall'] / (d['precision'] + d['recall'])
    
    return pd.Series(d)

In [ ]:
df_accuracy = df.groupby(['facilityCode', 'month']).apply(get_accuracy, codetype='EANDM').reset_index()
df_accuracy
pep_df = df.copy()

In [ ]:
df['serviceDate'] = pd.to_datetime(df['serviceDate'])

In [ ]:
set_cpt = set()
set_rejected = set()

In [ ]:
def get_cpt_counts(code_list, df_to_use, col, codetype):
    count_dict = {}
    for code in code_list:
        count_dict[code] = df_to_use[col].str.contains(code).sum()
        
    return count_dict

In [ ]:
def clean_cpt(x):
    
    clean = str(x['CPT_Final']).split(', ')
    for i in clean:
        i = i[0:5]
        set_cpt.add(i)
    return clean
cpt_code_list = set_cpt
clean_cpt = df.apply(clean_cpt, axis=1)

In [ ]:
pep_df[pep_df.EANDM_Final.isna()].shape

In [ ]:
count_dict_cpt = get_cpt_counts(cpt_code_list, pep_df[pep_df.EANDM_Final.isna()], 'CPT_Suggested', 'CPT_Suggested')

In [ ]:
cpt_stats = pd.DataFrame(count_dict_cpt.items(), columns = ['cpt_code','code_counts']).sort_values(by=['code_counts'], ascending=False).reset_index()

In [ ]:
cpt_stats.head(20)

In [ ]:
from ast import literal_eval
# from collections import Counter
# def get_counts(x):
#     d= Counter([c for y in x for c in literal_eval(y)])
    
#     for k, v in d.most_common():
#         print(k, v)
    # print(d)

In [ ]:
# get_counts(df[df.facilityCode==1475]['CPT_Accepted'])

In [ ]:
# get_counts(df[df.facilityCode=='PEP']['CPT_Rejected'])

In [ ]:
# get_counts(df[df.facilityCode=='PEP']['CPT_Added'])

In [ ]:
# df.facilityCode.value_counts()

In [ ]:
pep_df = df.copy()
# pep_df[pep_df['encDocIds'].fillna('').astype(str).apply(lambda x:len(literal_eval('['+str(x)+']'))==1)].shape, pep_df.shape

In [ ]:
# len(set(pep_df.id).intersection(set(enc_pcs_dict)))

In [ ]:
# pep_df['filepath'] = pep_df.id.apply(lambda x:enc_pcs_dict[x] if x in enc_pcs_dict else np.nan)
# pep_df = pep_df[~pep_df.filepath.isna()].copy()

In [ ]:
pep_df.EANDM_Final = pep_df.EANDM_Accepted.apply(lambda x:literal_eval(x))+pep_df.EANDM_Added.apply(lambda x:literal_eval(x))
pep_df.EANDM_Suggested = pep_df.EANDM_Accepted.apply(lambda x:literal_eval(x))+pep_df.EANDM_Rejected.apply(lambda x:literal_eval(x))

In [ ]:
def get_counts_group(x, grouped_col):
    print(x[grouped_col].unique())
    d= Counter([c for y in x['EANDM_Final'] for c in y])
    
    for k, v in d.most_common():
        print(k, v)
    print('\n')

In [ ]:
pep_df.EANDM_Final.value_counts()

In [ ]:
pep_df.EANDM_Suggested.value_counts()

In [ ]:
# for col in pep_df.columns:
#     print(col)
#     display(pep_df[col].value_counts())
#     print('\n\n')

In [ ]:
df.userId = df.userId.astype('str')
pep_df_1 = df[df.userId=='5050'].copy()

In [ ]:
pep_df_1

In [ ]:
n = pep_df_1.groupby(['userId','facilityCode']).apply(get_accuracy, 'EANDM')
n.to_excel('/home/trishalaswain/Documents/AGS_PROJECTS/download_scripts_and_tars/CHS_ALL_JUNE/CHS_userId_5050.xlsx')

In [ ]:
# # pep_df.groupby('physicianId').apply(get_counts_group, grouped_col='pointOfCare')
# pep_df_1 = df[df.EANDM_Final.isna()]
# pep_df_1.groupby(['visitType']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False])

In [ ]:
pep_df.EANDM_Final.value_counts()

In [ ]:
# pep_df = pep_df.groupby(['visitType']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False]).reset_index()
# df[df.visitType.fillna('').str.contains('Follow-up 15') & (~df.EANDM_Final.isna())][cols_to_use]
# temp[temp.encounters>10]

In [ ]:
# df.groupby(['payor','visitType']).apply(get_accuracy, 'EANDM').sort_values(['f1'],ascending=[True]).head(50).reset_index()

In [ ]:
# pep_df.groupby(['visitType']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False]).head(60).reset_index()

In [ ]:
# df[df.visitType.fillna('').str.contains('Follow-up 15')][cols_to_use].head(50)

In [ ]:
cols_to_use = ['physicianId', 'userId', 'month', 'visitType','accountNumber',
       'placeOfService', 'gender', 'pointOfCare','payor', 'EANDM_Final', 'EANDM_Suggested','CPT_Suggested', 'CPT_Final', 'HCPCS_Suggested', 'HCPCS_Final', 'CM_Suggested', 'CM_Final']

In [ ]:
df.shape

In [ ]:
df7 = df[df.CPT_Suggested.fillna('').str.contains('99024')]
# df7 = df[((df.visitType.fillna('').str.contains('STAPLE/STITCH REMOVAL')))]
# df7 = df[(df.CPT_Suggested.str.contains('57456')) | (df.CPT_Suggested.str.contains('58301'))]
# df.CPT_Suggested.str.contains('96372')) & 

# df7 = df[(df.CPT_Suggested.str.contains('0502F'))  & (df.EANDM_Final.isna())]
# df7 = df[((df.EANDM_Suggested.str.contains('99212')) & (~df.EANDM_Suggested.str.contains('99202'))) & df.CPT_Final.str.contains('20610')]
# df7 = df[~(df.CPT_Suggested.str.contains('99024')) & ~(df.CPT_Final.str.contains('99024'))& ((df.EANDM_Suggested.str.contains('99212'))) & (df.EANDM_Final=='0')]
# df7 = df[((df.EANDM_Suggested.str.contains('99212')) & (~df.EANDM_Suggested.str.contains('99202'))) & (df.EANDM_Final.isna())]

# df7 = df[(df.EANDM_Final.str.contains('99212') | df.EANDM_Final.str.contains('99213')) & (~df.EANDM_Suggested.str.contains('9'))]

# df7 = df[(df.CPT_Suggested.str.contains('99024')) & (df.EANDM_Suggested.str.contains('9'))]
# df7 = df[(df.visitType.str.contains('Established OB 15')) & ~(df.EANDM_Final=='0')]
# df7 = df[(df.visitType.str.contains('Established Patient 5')) & (df.pointOfCare.str.contains('OSSM_Sparks')) & (df.CPT_Final.str.contains('206'))]

# df7 = df[(df.pointOfCare.fillna('').str.contains('OSSM_Sparks'))]

df7[cols_to_use]



In [ ]:
df7[df7.CPT_Suggested.str.contains('0502F')].shape

In [ ]:
df7.CPT_Final = df7.CPT_Final.astype('str')

In [ ]:
df7[(~df.CPT_Final.str.contains("99024"))]

In [ ]:
df7.groupby(['visitType']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False])

In [ ]:
# df8[df8.EANDM_Final.isna()]

In [ ]:
df7[df7.EANDM_Final.isna()][cols_to_use]
count_dict_cpt = get_cpt_counts(cpt_code_list, df7[df7.EANDM_Final.isna()][cols_to_use], 'CPT_Suggested', 'CPT_Suggested')
cpt_stats = pd.DataFrame(count_dict_cpt.items(), columns = ['cpt_code','code_counts']).sort_values(by=['code_counts'], ascending=False).reset_index()

In [ ]:
cpt_stats.head()

In [ ]:
# df7.payor.value_counts()

In [ ]:
df7[df7.EANDM_Final.isna()][cols_to_use].shape

In [ ]:
df7[~df7.EANDM_Final.isna()][cols_to_use].shape

In [ ]:
# df7.groupby(['visitType']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False]).head(60).reset_index()

In [ ]:
df7[cols_to_use].EANDM_Final.value_counts()

In [ ]:
df7[cols_to_use].EANDM_Suggested.value_counts()

In [ ]:
df7[df7.EANDM_Final.isna()][cols_to_use]

In [ ]:
df7[~df7.EANDM_Final.isna()][cols_to_use]

In [ ]:
# df9[cols_to_use]

In [ ]:
df7.EANDM_Final.value_counts()

In [ ]:
df7.groupby(['visitType','pointOfCare']).apply(get_accuracy, 'EANDM').sort_values(['f1','EANDM_rejected'],ascending=[True,False]).reset_index()

In [ ]:
# new_df.sort_values('EANDM_rejected', ascending=False).reset_index()

In [ ]:
pep_df.columns
pep_df.CPT_Suggested = pep_df.CPT_Suggested.astype('str')
pep_df

In [ ]:
pep_df.CPT_Final = pep_df.CPT_Final.astype('str')

In [ ]:
df[df.CPT_Final.fillna('').str.contains('99024')]

In [ ]:
pep_df.EANDM_Final = pep_df.EANDM_Final.astype('str')

In [ ]:
pep = pep_df[pep_df.CPT_Suggested.str.contains('99024') & (~pep_df.EANDM_Final.fillna('').str.contains('9'))]

In [ ]:
pep.columns

In [ ]:
get_accuracy(pep, 'EANDM')

In [ ]:
pep[['accountNumber', 'EANDM_Final', 'EANDM_Suggested', 'CPT_Suggested', 'CPT_Final', 'CM_Suggested']]

In [ ]:
# visitTypes_to_ignore = ['VNG', 'FNA', 'Cochlear Implant 60', 'CLINIC Hearing Evaluation (30','Hearing Test','*Nurse Visit','*Procedure 15','Physical: Sports 15','Botox',]
# physicianIds_to_ignore = [28480,28485,10569]
# visitTypes_to_ignore = []
# df1 = pep_df[~pep_df.physicianId.isin(physicianIds_to_ignore)]
get_accuracy(pep_df[pep_df.CPT_Suggested.str.contains('99024')], 'EANDM')
# get_accuracy(pep_df[~pep_df.physicianId.isin(physicianIds_to_ignore)], 'EANDM')

In [ ]:
df.CPT_Suggested = df.CPT_Suggested.astype('str')
df.EANDM_Suggested = df.EANDM_Suggested.astype('str')
get_accuracy(df, 'EANDM')

In [ ]:
get_accuracy(df2, 'EANDM')

In [ ]:
for i, r in df2.iterrows():
    if '99024' in r['CPT_Suggested']:
        df2.at[i, 'EANDM_Rejected_Count'] = 0

In [ ]:
df2[df2.CPT_Suggested.str.contains('99024')].EANDM_Final.value_counts()

In [ ]:
df2[df2.CPT_Suggested.str.contains('99024')].EANDM_Suggested.value_counts()

In [ ]:
# df1 = pep_df[pep_df.physicianId.isin(physicianIds_to_ignore)].visitType.value_counts()

In [ ]:
get_accuracy(pep_df[~pep_df.CPT_Suggested.str.contains('99024')], 'EANDM')

In [ ]:
pep_df = pep_df[~pep_df.visitType.isin(visitTypes_to_ignore)]

In [ ]:
# pep_df[pep_df.pointOfCare=='ENT_MERIT HEALTH ENT'].groupby(['visitType']).apply(get_accuracy, 'EANDM').sort_values('f1').reset_index()

In [ ]:
pep_df.groupby(['payor']).apply(get_accuracy, 'EANDM').sort_values('EANDM_rejected',ascending=False).reset_index().head(15)

In [ ]:
pep_df.columns

In [ ]:
df4 = df[(df.CPT_Final.str.contains('57456')) | (df.CPT_Final.str.contains('58301'))]

In [ ]:
df4.EANDM_Final.value_counts()

In [ ]:
df4.groupby(['visitType','payor']).apply(get_accuracy, 'EANDM').sort_values('EANDM_billed',ascending=False).reset_index()

In [ ]:
df5 = df[df.visitType.str.contains('Botox')]

In [ ]:
df4.shape

In [ ]:
df6 = pd.concat([df4[cols_to_use],df5[cols_to_use]])

In [ ]:
df6.shape

In [ ]:
df6 = df6.drop_duplicates()

In [ ]:
df6

In [ ]:
cols_to_use = ['physicianId', 'userId', 'month', 'visitType','accountNumber',
       'placeOfService', 'gender', 'pointOfCare','payor', 'EANDM_Final', 'EANDM_Suggested','CPT_Final']
pep_df[pep_df.userId==6000][cols_to_use].T

In [ ]:
df5.EANDM_Final.value_counts()

In [ ]:
for facility in pep_df.facilityCode.unique():
    temp = pep_df[pep_df.facilityCode==facility]
    print(facility)
    display(temp.groupby('visitType').apply(get_accuracy, 'EANDM').sort_values('EANDM_rejected', ascending=False).reset_index())
    print()

In [ ]:
for facility in pep_df.facilityCode.unique():
    temp = pep_df[pep_df.facilityCode==facility]
    print(facility)
    display(temp.groupby('visitType').apply(get_accuracy, 'EANDM').sort_values('EANDM_rejected', ascending=False).reset_index())
    print()

In [ ]:
confusion_matrix = pd.crosstab(pep_df.EANDM_Final.astype(str), pep_df.EANDM_Suggested.astype(str))
confusion_matrix

In [ ]:
# from itertools import zip_longest

# def grouper(iterable, n, fillvalue=None):
#     args = [iter(iterable)] * n
#     return zip_longest(*args, fillvalue=fillvalue)

# # for x in grouper('ABCDEFG', 3, 'x'):
# #     print(x)
    
# def chunker(seq, size):
#     return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# for col in chunker(confusion_matrix.columns, 5):
#     # print(list(col))
#     print(col)
#     display(confusion_matrix[list(col)])
#     print()

In [ ]:
confusion_matrix.style.background_gradient(cmap='YlGnBu', vmin=-5, vmax=16.5)

In [ ]:
req_columns = ['accountNumber', 'gender', 'pointOfCare', 'visitType']
pep_df[(pep_df.EANDM_Suggested.astype(str)=='[99213, 99203]') & (pep_df.EANDM_Final.astype(str)=='[99213]')][req_columns+['EANDM_Suggested', 'EANDM_Final']]
# pep_df[(pep_df.EANDM_Suggested.astype(str)=='[99202, 99212]')][req_columns+['EANDM_Suggested', 'EANDM_Final']]

In [ ]:
def get_proper(x):
    if 99223 in x and (99238 in x or 99281 in x or 99283 in x or 99284 in x):
        return [99223]
    elif 99292 in x:
        return [99292]
    elif 99285 in x:
        return [99285]
    elif 99291 in x and (99283 in x or 99284 in x):
        return [99291]
    elif 99284 in x and (99283 in x or 99282 in x):
        return [99284]
    if 99418 in x:
        x.remove(99418)
    return x
pep_df.EANDM_Final.apply(get_proper).value_counts()

In [ ]:
pep_df.EANDM_Suggested.apply(get_proper).value_counts()

In [ ]:
pd.crosstab(pep_df.EANDM_Final.apply(get_proper).astype(str), 
            pep_df.EANDM_Suggested.apply(get_proper).astype(str)).style.background_gradient(cmap='YlGnBu', vmin=-5, vmax=16.5)

In [ ]:
type(pep_df.EANDM_Final[0])

In [ ]:
pep_df[pep_df.EANDM_Final.astype(str)=='[99223]'][req_columns]

In [ ]:
# df_accuracy_prev = df[df.serviceDate<'2023-01-01'].groupby(['facilityName', 'month']).apply(get_accuracy, codetype='CPT').reset_index()
# df_accuracy_new = df[df.serviceDate>='2023-01-01'].groupby(['facilityName', 'month']).apply(get_accuracy, codetype='CPT').reset_index()

In [ ]:
df_accuracy_new.head(30)

In [ ]:
df_accuracy_prev.head(30)

In [ ]:
def get_difference(data_df, codetype):
    
    d = OrderedDict()
    
    if data_df.shape[0]==2:
        jan = data_df[data_df.month=='2023-01'].iloc[0].to_dict()
        feb = data_df[data_df.month=='2023-02'].iloc[0].to_dict()
    
        d['diff_precision'] = feb['precision'] - jan['precision']
        d['diff_recall'] = feb['recall'] - jan['recall']
        d['diff_f1'] = feb['f1'] - jan['f1']
    
    else:
        d['diff_precision'] = 0.0
        d['diff_recall'] = 0.0
        d['diff_f1'] = 0.0

    return pd.Series(d)
                                                     
df_diff = df_accuracy.groupby('facilityName').apply(get_difference, codetype='CPT')

In [ ]:
df_accuracy[df_accuracy.facilityName=='NORTHWEST HEALTH-FAYETTEVILLE FSED']

In [ ]:
df.columns

In [ ]:
get_accuracy(df[df.month=='2023-01'], codetype='CPT')

In [ ]:
get_accuracy(df[df.month=='2023-02'], codetype='CPT')

In [ ]:
excluded_facilities = [
    'NORTHWEST HEALTH-FAYETTEVILLE FSED',
    'JASPER MEMORIAL HOSPITAL',
    'INDIAN PATH COMMUNITY HOSPITAL - PEDS',
    'HANCOCK COUNTY HOSPITAL - ICP',
    'THOREK MEMORIAL HOSPITAL-ANDERSONVILLE',
    'BIENVILLE MEDICAL CENTER',
]
# THOREK MEMORIAL HOSPITAL, HANCOCK COUNTY HOSPITAL - ICP, BAPTIST MEMORIAL HOSPITAL YAZOO - ICP,
# DICKENSON COMMUNITY HOSPITAL-ICP, BAPTIST MEMORIAL HOSPITAL CALHOUN - ICP,
# WINN PARISH MEDICAL CENTER, BIENVILLE MEDICAL CENTER, BRISTOL REGIONAL MEDICAL CENTER PEDS,


In [ ]:
get_accuracy(df[(df.month=='2023-02') & ~(df.facilityName.isin(excluded_facilities))], codetype='CPT')

In [ ]:
for name in df_diff.sort_values(['diff_precision'], ascending=[True]).reset_index().facilityName:
    print(df[df.facilityName==name].facilityCode.iloc[0])
    display(df_accuracy[df_accuracy.facilityName==name])
    print()

In [ ]:
for col in ['physicianId', 'userId', 'visitType', 'gender', 'pointOfCare', 'visitType.1', 'placeOfService.1', 
                'patientType', 'serviceLine', 'financialClass', 'payor']:
    print(col)        
    display(df[col].value_counts().reset_index())

In [ ]:
for name in df_diff.sort_values(['diff_precision'], ascending=[True]).reset_index().facilityName:
# for name in ['PIEDMONT CARTERSVILLE MEDICAL CENTER']:
    temp = df[df.facilityName==name]
    print(get_accuracy(temp[temp.month=='2023-01'], codetype='CPT'))
    print(get_accuracy(temp[temp.month=='2023-02'], codetype='CPT'))
    print(name)
    for col in ['physicianId', 'userId', 'processId', 'gender', 'financialClass', 'payor']:
        print(col)        
        display(temp[temp.month=='2023-01'][col].value_counts().reset_index())
        display(temp[temp.month=='2023-02'][col].value_counts().reset_index())
        print()
    print()
    print('----------------------------------------------------------')
    print()

In [ ]:
temp = df
get_accuracy(temp[(temp.facilityCode==1395) 
       & (temp.month=='2023-01')
           & (temp.serviceDate.str.contains('2023', na=False))
                 ], codetype='CPT')

In [ ]:
temp = df
get_accuracy(temp[(temp.facilityCode==1395) 
       & (temp.month=='2023-01')
           & (temp.serviceDate.str.contains('2022', na=False))
                 ], codetype='CPT')

In [ ]:
temp = df
get_accuracy(temp[(temp.facilityCode==1395) 
       & (temp.month=='2023-02')
           & (temp.serviceDate.str.contains('2023', na=False))
                 ], codetype='CPT')

In [ ]:
for eandmcode in ['99283', '99284', '99285', '99291']:
    print(eandmcode)
    temp = df[df.EANDM_Final.str.contains(eandmcode, na=False)]
    temp[(temp.facilityCode==1395) 
       & (temp.month=='2023-01') 
       & (temp.CPT_Suggested.str.contains('93010', na=False)) 
        & ~(temp.CPT_Final.str.contains('93010', na=False)) 
           & (temp.serviceDate.str.contains('2023', na=False)) 
       # & (df.EANDM_Final.str.contains('99282', na=False))
       # & (temp.refDocTypes.str.contains('EKG', na=False))
      ].sort_values('serviceDate').payor.value_counts().reset_index().apply(applytab, axis=1)
    # [['userId', 'processId', 'serviceDate', 'month', 'accountNumber',  'CPT_Final', 'CPT_Suggested', 'EANDM_Final', 'EANDM_Suggested']].apply(applytab,axis=1)
    print()
    print()

In [ ]:
for eandmcode in ['99283', '99284', '99285', '99291']:
    print(eandmcode)
    temp = df[df.EANDM_Final.str.contains(eandmcode, na=False)]
    temp[(temp.facilityCode==1395) 
       & (temp.month=='2023-01') 
       & (temp.CPT_Suggested.str.contains('93010', na=False)) 
       & (temp.CPT_Final.str.contains('93010', na=False)) 
         & (temp.serviceDate.str.contains('2023', na=False)) 
       # & (df.EANDM_Final.str.contains('99282', na=False))
       # & (temp.refDocTypes.str.contains('EKG', na=False))
      ].sort_values('serviceDate').payor.value_counts().reset_index().apply(applytab, axis=1)
    # [['userId', 'processId', 'serviceDate', 'month', 'accountNumber',  'CPT_Final', 'CPT_Suggested', 'EANDM_Final', 'EANDM_Suggested']].apply(applytab,axis=1)
    print()
    print()

In [ ]:
df[df.accountNumber==87340850][['userId', 'processId', 'serviceDate', 'month', 'accountNumber',  'CPT_Final', 'CPT_Suggested', 'EANDM_Final', 'EANDM_Suggested']]

In [ ]:
pd.crosstab(temp.EANDM_Suggested, temp.EANDM_Final).reset_index().T.reset_index().T.apply(applytab, axis=1);

In [ ]:
temp = df[(df.facilityCode==1395) 
       & (df.month=='2023-02')]

def get_new_counts(row):
    d = row.to_dict()
    if '99285' in row['EANDM_Suggested']:
        if '93010' in row['CPT_Accepted']:
            # display(row[['CPT_Final', 'EANDM_Final', 'EANDM_Suggested']])
            d['CPT_Accepted_Count'] -= 1
            d['CPT_Added_Count'] += 1
        elif '93010' in row['CPT_Rejected']:
            d['CPT_Rejected_Count'] -= 1
            
    return pd.Series(d)

temp_new_counts = temp.apply(get_new_counts,axis=1)

In [ ]:
temp.EANDM_Suggested.value_counts()

In [ ]:
temp.EANDM_Final.value_counts()

In [ ]:
display(get_accuracy(temp, codetype='CPT'))
display(get_accuracy(temp_new_counts, codetype='CPT'))